In [1]:
import pandas as pd
from google.cloud import bigquery
import requests
import time

In [2]:
query = """
SELECT tmdbId
FROM `film-wizard-453315.Grouplens.grouplens_links`
"""

In [3]:
client = bigquery.Client()
tmdb_list = client.query(query)
df = tmdb_list.to_dataframe()
df

/Users/oliverramsaygray/.pyenv/versions/3.10.6/envs/film_wizard/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,tmdbId
0,2
1,3
2,5
3,6
4,11
...,...
87456,1179468
87457,1181568
87458,1181806
87459,1182286


In [9]:
movie_credits_list = []

# Loop through the desired rows
for i in range(87300, 87310):    
    if pd.notna(df.iloc[i, 0]):  # Skip if NA
        tmdbId = int(df.iloc[i, 0])
        
        url = f"https://api.themoviedb.org/3/movie/{tmdbId}/credits?language=en-US"
        
        headers = {
            "accept": "application/json",
            "Authorization": "Bearer <insert key>"
        }
        
        # Fetch the movie credits data
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            # Parse JSON response
            movie_credits_data = response.json()

            # Extract actor names (only first 5 actors if available)
            cast = movie_credits_data.get("cast", [])
            actors = {f"actor_{i+1}": cast[i]["name"] for i in range(min(5, len(cast)))}

            # Filter crew members with specific jobs
            crew = movie_credits_data.get("crew", [])
            filtered_crew = {}
            for person in crew:
                if person.get("job") in ['Director', 'Writer', 'Producer', 'Executive Producer', 'Director of Photography', 'Editor', 'Original Music Composer']:
                    # Avoid overwriting if multiple people have the same job
                    job_title = person["job"]
                    if job_title in filtered_crew:
                        filtered_crew[job_title] += f", {person['name']}"
                    else:
                        filtered_crew[job_title] = person["name"]

            # Combine movie details, actors, and filtered crew into a single dictionary
            movie_credits = {**actors, **filtered_crew}
            movie_credits_list.append(movie_credits)

# Convert the list of dictionaries into a pandas DataFrame
movie_credits_df = pd.DataFrame(movie_credits_list)

# Display the DataFrame with all columns shown
pd.set_option('display.max_columns', None)
movie_credits_df

,actor_1,actor_2,actor_3,actor_4,actor_5,Director,Writer,Producer,Editor,Director of Photography,Executive Producer,Original Music Composer
0,Le Phong Vu,Nguyen Thinh,Nguyen Thi Truc Quynh,Vu Ngoc Manh,Dylan Besseau,Phạm Thiên Ân,Phạm Thiên Ân,"Tran Van Thi, Jeremy Chua",Phạm Thiên Ân,Dinh Duy Hung,Tran Van Thi,NaN
1,Simone Buttelli,NaN,NaN,NaN,NaN,Maria Augusta V. Nunes,Maria Augusta V. Nunes,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Henry Roosevelt,NaN,NaN,"Jawad Metni, Pax Wassermann",Patrick Ginnetty,"Liz Garbus, Jon Bardin, Kate Barry, Dan Cogan",NaN
3,Manolo Cardona,Maribel Verdú,Carla Adell,Juan Carlos Remolina,Adriana Paz,Manolo Cardona,NaN,NaN,Camilo Abadía,Luis Enrique Carrión,NaN,NaN
4,Gayle Rankin,Hari Nef,Annabelle Dexter-Jones,Rad Pereira,Jared Abrahamson,Stewart Thorndike,Stewart Thorndike,"Lizzie Shapiro, Lexi Tannenholtz","Kathryn J. Schubert, Thomas Emmet Ashton",Grant Greenberg,"Samuel Zimmerman, Nicholas Lazo, Emily Gotto",Jason Falker
5,River Gallo,Alicia Roth Weigel,Sean Saifa Wall,Julie Cohen,NaN,Julie Cohen,NaN,"Molly O'Brien, Tommy Nguyen",Kelly Kendrick,"Amy Bench, Kate Phelan, Leah Anova","Andy Berg, Liz Cole, Elizabeth Fischer, Noah O...",Amanda Yamate
6,Rock Hudson,Joe Carberry,Tim Turner,Lee Garlington,Paul Garlington,Stephen Kijak,NaN,"Sarah Schechter, Will Clarke, Greg Berlanti, G...",Claire Didier,NaN,"Mike Runagall, Michael McGrath, Andy Mayson, N...",Laura Karpman
7,Lovell Gates,Jakkar Thompson,Clayton B. Stevens,NaN,NaN,Sam Pollard,NaN,"Dave Sirulnick, Byron Motley, Jen Isaacson, Ro...",Dave Marcus,"Henry Adebonojo, Angel Barroeta","Jeffrey Lurie, Jon Kamen, Todd Wagner, Shawn G...",NaN
8,Jeff Daniels,Charlie Day,Mike Veeck,Libby Veeck,Night Train Veeck,"Morgan Neville, Jeff Malmberg",NaN,"Morgan Neville, Jon Berg, Danny Breen","Alan Lowe, Jeff Malmberg",Antonio Cisneros,"Caitrin Rogers, Greg Silverman, Fran Zeuli","Daniel Wohl, Garth Neustadter"
9,Bill Marler,Darin Detwiler,Dr. John Kobayashi,"Lance Price, Ph.D.",Robert Nugent,Stephanie Soechtig,Jeff Benedict,"Ross M. Dinerstein, Kristin Lazure",Weston Cadwell,NaN,"Jeff Benedict, Ross Girard, Rebecca Evans",NaN


In [5]:
# List to store movie credits data
movie_credits_list = []

# Define the movie ID and API URL
url = "https://api.themoviedb.org/3/movie/27205/credits?language=en-US"
headers = {
    "accept": "application/json",
    "Authorization": "Bearer <insert key againn>"
}

# Fetch the movie credits data
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Parse JSON response
    movie_credits_data = response.json()

    # Extract actor names (only first 5 actors) and add them as separate columns
    actors = {f"actor_{i+1}_name": movie_credits_data["cast"][i]["name"] for i in range(5)}

    # Filter crew members with specific jobs and add them as separate columns with job titles as column names
    crew = movie_credits_data.get("crew", [])
    filtered_crew = {}
    for person in crew:
        if person["job"] in ['Director', 'Writer', 'Producer', 'Executive Producer', 'Director of Photography']:
            # Use job title as column name and assign the person's name as the value
            filtered_crew[person["job"]] = person["name"]

    # Combine movie details, actors, and filtered crew into a single dictionary for each movie
    movie_credits = {**actors, **filtered_crew}
    movie_credits_list.append(movie_credits)

# Convert the list of dictionaries into a pandas DataFrame
movie_credits_df = pd.DataFrame(movie_credits_list)

# Display the DataFrame with all columns shown
pd.set_option('display.max_columns', None)
movie_credits_df

,actor_1_name,actor_2_name,actor_3_name,actor_4_name,actor_5_name,Producer,Executive Producer,Director of Photography,Writer,Director
0,Leonardo DiCaprio,Joseph Gordon-Levitt,Ken Watanabe,Tom Hardy,Elliot Page,Luke Mendes,Chris Brigham,Wally Pfister,Christopher Nolan,Christopher Nolan


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [8]:
# Create a dictionary with user ratings for 5 films
data = {
    "Film A": [5, 3, 4, 2, 1],
    "Film B": [4, 2, 5, 3, 2],
    "Film C": [3, 5, 2, 4, 3],
    "Film D": [1, 4, 3, 5, 4],
    "Film E": [2, 1, 4, 3, 5],
}

# Convert to DataFrame with user labels
df = pd.DataFrame(data, index=[f"User {i+1}" for i in range(5)])
df

,Film A,Film B,Film C,Film D,Film E
User 1,5,4,3,1,2
User 2,3,2,5,4,1
User 3,4,5,2,3,4
User 4,2,3,4,5,3
User 5,1,2,3,4,5


In [23]:
X = df.iloc[0, :].values.reshape(1, -1)  # Convert to 2D
y = df.iloc[1:, :].values  # Keep y as a 2D array

In [24]:
cosine_similarity(X, y)

array([[0.8       , 0.91863818, 0.76447079, 0.65454545]])

NameError: name 'df_raw' is not defined